In [1]:
import torch.nn as nn
import torchvision
import torch.utils.data
import torch.optim as optim
from models.big_parking_net import BigParkingNet

In [2]:
# Dataset loading
folder_train = torchvision.datasets.ImageFolder(
    root="../train_images",
    transform=torchvision.transforms.Compose([
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        torchvision.transforms.ToTensor()
    ])
)

train_loader = torch.utils.data.DataLoader(
    folder_train,
    batch_size=64,
    shuffle=True
)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.BCEWithLogitsLoss()
model = BigParkingNet().to(device)
model.load("../trained/big_parking_net2.pth")
model.train()
optimizer = optim.AdamW(model.parameters(), lr=0.00001)

In [7]:
# Train
num_episodes = 5

best_loss = float("inf")
for episode in range(num_episodes):
    episode_loss = 0

    for images, labels in train_loader:
        # Preprocess
        images, labels = images.to(device), labels.float().to(device).view(-1, 1)

        # Reset gradients
        optimizer.zero_grad()

        # Feed stuff into model
        outputs = model(images)

        # Compute loss and backpropagate
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Accumulate loss
        episode_loss += loss.item()

    print(f"Episode {episode}, loss: {episode_loss}")

    if episode_loss < best_loss:
        best_loss = episode_loss
        torch.save(model.state_dict(), "../trained/big_parking_net3.pth")
        print("\tSaving!")


Episode 0, loss: 0.045548901025540545
	Saving!
Episode 1, loss: 0.016156061174115166
	Saving!
Episode 2, loss: 0.016075917836133158
	Saving!
Episode 3, loss: 0.007098491431861476
	Saving!
Episode 4, loss: 0.007390448595288035
